In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import torch

print(torch.cuda.is_available())
#True
print(torch.cuda.device_count())
#1
print(torch.cuda.current_device())
#0
print(torch.cuda.get_device_name(0))
#'GeForce GTX 1080'

True
1
0
NVIDIA GeForce RTX 3090


In [7]:
from __future__ import annotations
import functools
import typing as tp
import datasets
import transformers
from transformers import (
    DataCollatorForSeq2Seq,
    PreTrainedTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)


increment_en = [
    {"input": "One", "target": "Two"},
    {"input": "Three", "target": "Four"},
    {"input": "Five", "target": "Six"},
    {"input": "Seven", "target": "Eight"},
    {"input": "Nine", "target": "Ten"},
]
increment_en = increment_en * 100


def lod_to_dol(list_of_dicts: tp.List[tp.Dict[str, tp.Any]]) -> tp.Dict[str, list]:
    dict_of_lists = {
        key: [dct[key] for dct in list_of_dicts] for key in list_of_dicts[0]
    }
    return dict_of_lists


increment_en = lod_to_dol(increment_en)


def preprocess_function_(
    examples,
    tokenizer: PreTrainedTokenizer,
    max_input_length: int,
    max_target_length: int,
):
    inputs = examples["input"]
    targets = examples["target"]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


def preprocess_function2_(
    examples,
    tokenizer: PreTrainedTokenizer,
    max_input_length: int,
    max_target_length: int,
):
    inputs = examples["inputs"]
    targets = examples["labels"]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenizer = transformers.T5TokenizerFast.from_pretrained("google/flan-t5-small")
model = transformers.T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

args = Seq2SeqTrainingArguments(
    "script_debug",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    fp16=False,
    push_to_hub=False,
    # sharded_ddp=["zero_dp_3"],
    max_steps=114611,
    logging_steps=1000,
    save_steps=5000
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

dataset = datasets.DatasetDict(
    {
        "train": datasets.load_from_disk("woz_train_concat.dataset"),
        "test": datasets.load_from_disk("woz_test_concat.dataset"),
    }
)

preprocess_function2 = functools.partial(
    preprocess_function2_,
    tokenizer=tokenizer,
    max_input_length=512,
    max_target_length=512
)

processed_ds2 = dataset.map(preprocess_function2, batched=True)
processed_ds2.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=processed_ds2["train"],
    eval_dataset=processed_ds2["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

Map:   0%|          | 0/114611 [00:00<?, ? examples/s]

/home/sherman/miniconda3/envs/fn_env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/12760 [00:00<?, ? examples/s]

/home/sherman/miniconda3/envs/fn_env/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1000,2.657900
2000,2.428900
3000,2.331000
4000,2.257000
5000,2.209900
6000,2.168300
7000,2.155900
8000,2.128100
9000,2.111400
10000,2.065800


In [12]:
model.generate(**(tokenizer("One", return_tensors="pt").to(model.device)))

tensor([[   0, 2759,    1]], device='cuda:0')

In [10]:
pipe = transformers.pipelines.Text2TextGenerationPipeline(model=model, tokenizer=tokenizer, device=model.device, batch_size=4)

In [22]:
pipe("what")#, do_sample=True, eta_cutoff=3e-4)

[{'generated_text': "I'm looking for a place to eat lunch in Sacramento, California."}]

wandb: Network error (ReadTimeout), entering retry loop.
